In [1]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
df=pd.read_csv('C:/Users/jira_/Desktop/new_stresult17-13-17.csv',header=None)
d = df.values
ntotal=len(df)

# we do not renormalize it 
#d = normalize(d, axis=0, norm='l2')


resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(8)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,7):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 14
n_encoder_h_2 = 9



#n_encoder_h_5 = 10


# 3 hidden layers for decoder
#n_decoder_h_1 = 10
n_decoder_h_1 = 14
n_decoder_h_2 = 19




# Parameters
learning_rate = 0.005

#batch_size = 7
display_step = 1

total_batch=6
training_epochs = 5000

In [2]:
#we store the variables here
log_files_path = 'C:/Users/jira_/Desktop/new_st13-9-13/tmp/model.ckpt'
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        
        with tf.variable_scope("code"):
            output = layer(x, [13, n_code], [n_code], phase_train)


    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
   
        with tf.variable_scope("output"):
            output = layer(x, [ n_code, 13], [13], phase_train)

    return output
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')

    #if a python file, please use the 4 lines bellow and comment the "n_code = '2'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '9'
    
    #feel free to change with your own 
    #log_files_path = r'C:\Users\yy2895\Desktop\pys'
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 13]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive2=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, 'C:/Users/jira_/Desktop/new_st13-9-13/tmp/model.ckpt')
                        togive8=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,13)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive8.append(output_any_image)
                            
                        togive2=togive8
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)
            

we begin
Epoch: 0001 cost = 1.127001842
Validation Loss: 1.7327813
Epoch: 0002 cost = 1.113284171
Validation Loss: 1.3326257
Epoch: 0003 cost = 1.099869947
Validation Loss: 1.171539
Epoch: 0004 cost = 1.086784224
Validation Loss: 1.0866731
Epoch: 0005 cost = 1.073764622
Validation Loss: 1.020414
Epoch: 0006 cost = 1.060347895
Validation Loss: 0.9901744
Epoch: 0007 cost = 1.046315054
Validation Loss: 0.9921994
Epoch: 0008 cost = 1.031999052
Validation Loss: 1.0797842
Epoch: 0009 cost = 1.018245975
Validation Loss: 1.1308783
Epoch: 0010 cost = 1.005804201
Validation Loss: 1.1342486
Epoch: 0011 cost = 0.994639923
Validation Loss: 1.0963963
Epoch: 0012 cost = 0.984456033
Validation Loss: 1.0532764
Epoch: 0013 cost = 0.975154291
Validation Loss: 1.0266117
Epoch: 0014 cost = 0.966680666
Validation Loss: 1.0081892
Epoch: 0015 cost = 0.958865573
Validation Loss: 0.97082776
Epoch: 0016 cost = 0.951486776
Validation Loss: 0.84322864
Epoch: 0017 cost = 0.944354077
Validation Loss: 0.81839395
Epoc

Epoch: 0143 cost = 0.310420762
Validation Loss: 0.34123534
Epoch: 0144 cost = 0.307579319
Validation Loss: 0.3257898
Epoch: 0145 cost = 0.304756626
Validation Loss: 0.33298928
Epoch: 0146 cost = 0.301954413
Validation Loss: 0.34554395
Epoch: 0147 cost = 0.299170852
Validation Loss: 0.34366202
Epoch: 0148 cost = 0.296406552
Validation Loss: 0.32505837
Epoch: 0149 cost = 0.293661396
Validation Loss: 0.29264146
Epoch: 0150 cost = 0.290935010
Validation Loss: 0.30340737
Epoch: 0151 cost = 0.288228065
Validation Loss: 0.30325043
Epoch: 0152 cost = 0.285539235
Validation Loss: 0.29917458
Epoch: 0153 cost = 0.282869284
Validation Loss: 0.2896506
Epoch: 0154 cost = 0.280217389
Validation Loss: 0.32040435
Epoch: 0155 cost = 0.277584692
Validation Loss: 0.30744457
Epoch: 0156 cost = 0.274969901
Validation Loss: 0.29314488
Epoch: 0157 cost = 0.272373125
Validation Loss: 0.2753031
Epoch: 0158 cost = 0.269795239
Validation Loss: 0.27936074
Epoch: 0159 cost = 0.267234892
Validation Loss: 0.2672241
E

Epoch: 0291 cost = 0.058532891
Validation Loss: 0.0788401
Epoch: 0292 cost = 0.057711746
Validation Loss: 0.08519489
Epoch: 0293 cost = 0.056899686
Validation Loss: 0.08321543
Epoch: 0294 cost = 0.056096603
Validation Loss: 0.07943517
Epoch: 0295 cost = 0.055302654
Validation Loss: 0.09455023
Epoch: 0296 cost = 0.054517411
Validation Loss: 0.0825492
Epoch: 0297 cost = 0.053741358
Validation Loss: 0.07371419
Epoch: 0298 cost = 0.052974205
Validation Loss: 0.08537163
Epoch: 0299 cost = 0.052216066
Validation Loss: 0.08174409
Epoch: 0300 cost = 0.051466562
Validation Loss: 0.083633006
Epoch: 0301 cost = 0.050726650
Validation Loss: 0.10205425
Epoch: 0302 cost = 0.049995262
Validation Loss: 0.123088695
Epoch: 0303 cost = 0.049273025
Validation Loss: 0.14847317
Epoch: 0304 cost = 0.048559921
Validation Loss: 0.092283264
Epoch: 0305 cost = 0.047855496
Validation Loss: 0.077190526
Epoch: 0306 cost = 0.047160585
Validation Loss: 0.0511582
Epoch: 0307 cost = 0.046474366
Validation Loss: 0.07725

Validation Loss: 0.04599362
Epoch: 0453 cost = 0.017354724
Validation Loss: 0.055845067
Epoch: 0454 cost = 0.017340375
Validation Loss: 0.06519687
Epoch: 0455 cost = 0.017325991
Validation Loss: 0.04427389
Epoch: 0456 cost = 0.017311613
Validation Loss: 0.043412354
Epoch: 0457 cost = 0.017297489
Validation Loss: 0.052705467
Epoch: 0458 cost = 0.017283639
Validation Loss: 0.049851738
Epoch: 0459 cost = 0.017269933
Validation Loss: 0.049653683
Epoch: 0460 cost = 0.017256261
Validation Loss: 0.044679727
Epoch: 0461 cost = 0.017242627
Validation Loss: 0.055780638
Epoch: 0462 cost = 0.017229343
Validation Loss: 0.051241457
Epoch: 0463 cost = 0.017216323
Validation Loss: 0.042984303
Epoch: 0464 cost = 0.017202932
Validation Loss: 0.034220964
Epoch: 0465 cost = 0.017190121
Validation Loss: 0.035412993
Epoch: 0466 cost = 0.017177219
Validation Loss: 0.037878975
Epoch: 0467 cost = 0.017164419
Validation Loss: 0.04293175
Epoch: 0468 cost = 0.017151587
Validation Loss: 0.034925062
Epoch: 0469 cos

Epoch: 0592 cost = 0.016056787
Validation Loss: 0.052775025
Epoch: 0593 cost = 0.016050234
Validation Loss: 0.054575108
Epoch: 0594 cost = 0.016043768
Validation Loss: 0.059806686
Epoch: 0595 cost = 0.016037407
Validation Loss: 0.06682348
Epoch: 0596 cost = 0.016030901
Validation Loss: 0.07514614
Epoch: 0597 cost = 0.016024170
Validation Loss: 0.05119134
Epoch: 0598 cost = 0.016017936
Validation Loss: 0.057853352
Epoch: 0599 cost = 0.016011470
Validation Loss: 0.046904676
Epoch: 0600 cost = 0.016004853
Validation Loss: 0.055064525
Epoch: 0601 cost = 0.015998689
Validation Loss: 0.051015615
Epoch: 0602 cost = 0.015992236
Validation Loss: 0.035454594
Epoch: 0603 cost = 0.015985970
Validation Loss: 0.046574533
Epoch: 0604 cost = 0.015979789
Validation Loss: 0.04848472
Epoch: 0605 cost = 0.015973427
Validation Loss: 0.045381505
Epoch: 0606 cost = 0.015967124
Validation Loss: 0.042040262
Epoch: 0607 cost = 0.015960784
Validation Loss: 0.036509614
Epoch: 0608 cost = 0.015954706
Validation Lo

...

Epoch: 4312 cost = 0.011738875
Validation Loss: 0.092107475
Epoch: 4313 cost = 0.011773564
Validation Loss: 0.09723275
Epoch: 4314 cost = 0.011778130
Validation Loss: 0.07714975
Epoch: 4315 cost = 0.011761882
Validation Loss: 0.07033542
Epoch: 4316 cost = 0.011737358
Validation Loss: 0.08166724
Epoch: 4317 cost = 0.011713110
Validation Loss: 0.09099548
Epoch: 4318 cost = 0.011692027
Validation Loss: 0.077228986
Epoch: 4319 cost = 0.011675326
Validation Loss: 0.0892578
Epoch: 4320 cost = 0.011663065
Validation Loss: 0.09008689
Epoch: 4321 cost = 0.011654432
Validation Loss: 0.07831315
Epoch: 4322 cost = 0.011647547
Validation Loss: 0.06802916
Epoch: 4323 cost = 0.011641089
Validation Loss: 0.08071111
Epoch: 4324 cost = 0.011634042
Validation Loss: 0.0862469
Epoch: 4325 cost = 0.011625012
Validation Loss: 0.081041895
Epoch: 4326 cost = 0.011615614
Validation Loss: 0.068782814
Epoch: 4327 cost = 0.011605384
Validation Loss: 0.044835307
Epoch: 4328 cost = 0.011595041
Validation Loss: 0.031

Epoch: 4457 cost = 0.011637395
Validation Loss: 0.036168773
Epoch: 4458 cost = 0.011627024
Validation Loss: 0.032629784
Epoch: 4459 cost = 0.011619685
Validation Loss: 0.03178102
Epoch: 4460 cost = 0.011613967
Validation Loss: 0.030785955
Epoch: 4461 cost = 0.011608520
Validation Loss: 0.045447156
Epoch: 4462 cost = 0.011600944
Validation Loss: 0.04317155
Epoch: 4463 cost = 0.011592843
Validation Loss: 0.02802145
Epoch: 4464 cost = 0.011582619
Validation Loss: 0.038400803
Epoch: 4465 cost = 0.011572140
Validation Loss: 0.045766726
Epoch: 4466 cost = 0.011561556
Validation Loss: 0.050426163
Epoch: 4467 cost = 0.011552092
Validation Loss: 0.044740334
Epoch: 4468 cost = 0.011542312
Validation Loss: 0.063716784
Epoch: 4469 cost = 0.011534245
Validation Loss: 0.05451211
Epoch: 4470 cost = 0.011526456
Validation Loss: 0.051116597
Epoch: 4471 cost = 0.011520087
Validation Loss: 0.040911514
Epoch: 4472 cost = 0.011515097
Validation Loss: 0.04203432
Epoch: 4473 cost = 0.011512289
Validation Los

Validation Loss: 0.08352248
Epoch: 4603 cost = 0.011574295
Validation Loss: 0.04313508
Epoch: 4604 cost = 0.011567644
Validation Loss: 0.05061286
Epoch: 4605 cost = 0.011558993
Validation Loss: 0.055301644
Epoch: 4606 cost = 0.011549332
Validation Loss: 0.05745505
Epoch: 4607 cost = 0.011539192
Validation Loss: 0.07212041
Epoch: 4608 cost = 0.011528822
Validation Loss: 0.086991146
Epoch: 4609 cost = 0.011519040
Validation Loss: 0.08977535
Epoch: 4610 cost = 0.011509922
Validation Loss: 0.081488766
Epoch: 4611 cost = 0.011501298
Validation Loss: 0.057517078
Epoch: 4612 cost = 0.011493486
Validation Loss: 0.03511413
Epoch: 4613 cost = 0.011486849
Validation Loss: 0.034731995
Epoch: 4614 cost = 0.011481301
Validation Loss: 0.04566604
Epoch: 4615 cost = 0.011477000
Validation Loss: 0.051152773
Epoch: 4616 cost = 0.011474960
Validation Loss: 0.041024353
Epoch: 4617 cost = 0.011476697
Validation Loss: 0.067252144
Epoch: 4618 cost = 0.011482682
Validation Loss: 0.083964705
Epoch: 4619 cost = 

Epoch: 4750 cost = 0.011536112
Validation Loss: 0.046219703
Epoch: 4751 cost = 0.011528426
Validation Loss: 0.057266925
Epoch: 4752 cost = 0.011520341
Validation Loss: 0.08452291
Epoch: 4753 cost = 0.011510835
Validation Loss: 0.055659648
Epoch: 4754 cost = 0.011501212
Validation Loss: 0.045673933
Epoch: 4755 cost = 0.011491741
Validation Loss: 0.04696032
Epoch: 4756 cost = 0.011482721
Validation Loss: 0.05576253
Epoch: 4757 cost = 0.011474340
Validation Loss: 0.045175564
Epoch: 4758 cost = 0.011466085
Validation Loss: 0.03157335
Epoch: 4759 cost = 0.011458880
Validation Loss: 0.048227586
Epoch: 4760 cost = 0.011452468
Validation Loss: 0.0447219
Epoch: 4761 cost = 0.011446970
Validation Loss: 0.029292572
Epoch: 4762 cost = 0.011443252
Validation Loss: 0.027824426
Epoch: 4763 cost = 0.011441447
Validation Loss: 0.048509225
Epoch: 4764 cost = 0.011442932
Validation Loss: 0.04918076
Epoch: 4765 cost = 0.011448959
Validation Loss: 0.036815707
Epoch: 4766 cost = 0.011460179
Validation Loss:

Epoch: 4895 cost = 0.011534536
Validation Loss: 0.027003855
Epoch: 4896 cost = 0.011529106
Validation Loss: 0.030909292
Epoch: 4897 cost = 0.011523084
Validation Loss: 0.04055628
Epoch: 4898 cost = 0.011517287
Validation Loss: 0.074625805
Epoch: 4899 cost = 0.011511923
Validation Loss: 0.082472935
Epoch: 4900 cost = 0.011506127
Validation Loss: 0.06884055
Epoch: 4901 cost = 0.011500135
Validation Loss: 0.03950409
Epoch: 4902 cost = 0.011494295
Validation Loss: 0.046700876
Epoch: 4903 cost = 0.011487915
Validation Loss: 0.038088497
Epoch: 4904 cost = 0.011480579
Validation Loss: 0.026916726
Epoch: 4905 cost = 0.011473159
Validation Loss: 0.019795684
Epoch: 4906 cost = 0.011465316
Validation Loss: 0.030153403
Epoch: 4907 cost = 0.011456769
Validation Loss: 0.016913215
Epoch: 4908 cost = 0.011448353
Validation Loss: 0.032803588
Epoch: 4909 cost = 0.011440340
Validation Loss: 0.045439817
Epoch: 4910 cost = 0.011433031
Validation Loss: 0.025440475
Epoch: 4911 cost = 0.011425787
Validation L

In [3]:
import csv
with open('C:/Users/jira_/Desktop/new_stresult13-9-13.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive2)):
        wr.writerow(list(togive2[i][0]))